In [6]:
pip install dask

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------- ----------------------- 0.5/1.2 MB 10.7 MB/s eta 0:00:01
   --------------------------------- ------ 1.0/1.2 MB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 8.7 MB/s eta 0:00:00


DEPRECATION: Loading egg at c:\users\mubashir khan\appdata\local\programs\python\python312\lib\site-packages\spylon_kernel-0+unknown-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# File paths
sales_file_path = r'C:\Users\MUBASHIR KHAN\Desktop\jupyter\DMV\sales.csv'
product_file_path = r'C:\Users\MUBASHIR KHAN\Desktop\jupyter\DMV\product_hierarchy.csv'
store_file_path = r'C:\Users\MUBASHIR KHAN\Desktop\jupyter\DMV\store_cities.csv'

# Load datasets with proper column types
dtype_dict_sales = {'Product ID': 'str', 'Store ID': 'str', 'Sales Amount': 'float64'}
dtype_dict_product = {'Product ID': 'str'}
dtype_dict_store = {'Store ID': 'str'}

def load_data(file_path, dtype_dict=None):
    try:
        return pd.read_csv(file_path, dtype=dtype_dict, low_memory=False)
    except MemoryError:
        print("MemoryError: Unable to load the file.")
        return None

# Load datasets
sales_df = load_data(sales_file_path, dtype_dict_sales)
product_df = load_data(product_file_path, dtype_dict_product)
store_df = load_data(store_file_path, dtype_dict_store)

# Print the first few rows and column names
print("Sales DataFrame columns:", sales_df.columns)
print("Product DataFrame columns:", product_df.columns)
print("Store DataFrame columns:", store_df.columns)

# Print the first few rows of each DataFrame
print(sales_df.head())
print(product_df.head())
print(store_df.head())

# Print dataset info
print(sales_df.info())
print(product_df.info())
print(store_df.info())

# Check if the required columns are present
required_columns_sales = {'Product ID', 'Store ID', 'Sales Amount'}
required_columns_product = {'Product ID'}
required_columns_store = {'Store ID'}

print("Sales DataFrame columns missing:", required_columns_sales - set(sales_df.columns))
print("Product DataFrame columns missing:", required_columns_product - set(product_df.columns))
print("Store DataFrame columns missing:", required_columns_store - set(store_df.columns))

# Adjust column names if necessary (example)
# sales_df.rename(columns={'Product ID ': 'Product ID'}, inplace=True)  # Adjust if needed

# Merge datasets
try:
    sales_product_df = pd.merge(sales_df, product_df, on='Product ID', how='left')
    sales_product_store_df = pd.merge(sales_product_df, store_df, on='Store ID', how='left')

    # Check the merged dataset
    print(sales_product_store_df.head())
    print(sales_product_store_df.info())

    # Group by region and calculate total sales amount
    sales_by_region = sales_product_store_df.groupby('Region')['Sales Amount'].sum().reset_index()
    sales_by_region = sales_by_region.sort_values(by='Sales Amount', ascending=False)

    # Bar plot for sales distribution by region
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Region', y='Sales Amount', data=sales_by_region, palette='viridis')
    plt.title('Total Sales Amount by Region')
    plt.xlabel('Region')
    plt.ylabel('Total Sales Amount')
    plt.xticks(rotation=45)
    plt.show()

    # Pie chart for sales distribution by region
    plt.figure(figsize=(8, 8))
    plt.pie(sales_by_region['Sales Amount'], labels=sales_by_region['Region'], autopct='%1.1f%%', colors=sns.color_palette('viridis', len(sales_by_region)))
    plt.title('Sales Distribution by Region')
    plt.show()

    # Identify top-performing regions
    top_regions = sales_by_region.head(5)
    print("Top Performing Regions:")
    print(top_regions)

    # Group by region and product category
    sales_by_region_category = sales_product_store_df.groupby(['Region', 'Product Category'])['Sales Amount'].sum().reset_index()

    # Pivot the data for better visualization
    sales_pivot = sales_by_region_category.pivot(index='Region', columns='Product Category', values='Sales Amount').fillna(0)
    print(sales_pivot)

    # Stacked bar plot
    sales_pivot.plot(kind='bar', stacked=True, figsize=(12, 8), colormap='viridis')
    plt.title('Sales Amount by Region and Product Category (Stacked)')
    plt.xlabel('Region')
    plt.ylabel('Total Sales Amount')
    plt.xticks(rotation=45)
    plt.legend(title='Product Category')
    plt.show()

    # Grouped bar plot
    sales_pivot.plot(kind='bar', figsize=(12, 8), colormap='viridis')
    plt.title('Sales Amount by Region and Product Category (Grouped)')
    plt.xlabel('Region')
    plt.ylabel('Total Sales Amount')
    plt.xticks(rotation=45)
    plt.legend(title='Product Category')
    plt.show()

except KeyError as e:
    print(f"KeyError: {e}")

Sales DataFrame columns: Index(['product_id', 'store_id', 'date', 'sales', 'revenue', 'stock', 'price',
       'promo_type_1', 'promo_bin_1', 'promo_type_2', 'promo_bin_2',
       'promo_discount_2', 'promo_discount_type_2'],
      dtype='object')
Product DataFrame columns: Index(['product_id', 'product_length', 'product_depth', 'product_width',
       'cluster_id', 'hierarchy1_id', 'hierarchy2_id', 'hierarchy3_id',
       'hierarchy4_id', 'hierarchy5_id'],
      dtype='object')
Store DataFrame columns: Index(['store_id', 'storetype_id', 'store_size', 'city_id'], dtype='object')
  product_id store_id        date  sales  revenue  stock  price promo_type_1  \
0      P0001    S0002  2017-01-02    0.0     0.00    8.0   6.25         PR14   
1      P0001    S0012  2017-01-02    1.0     5.30    0.0   6.25         PR14   
2      P0001    S0013  2017-01-02    2.0    10.59    0.0   6.25         PR14   
3      P0001    S0023  2017-01-02    0.0     0.00    6.0   6.25         PR14   
4      P0001  